<a href="https://colab.research.google.com/github/RamMohan112/Assignement_J/blob/master/Solution_Problem_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np

import matplotlib.pyplot as plt


In [ ]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
## For Copying Data From Drive to Colab Instance Important for Faster Training Loop

!cp -r "/content/drive/My Drive/Colab Notebooks/Solution_IIT_Jodhpur/GurNum/" new_dir

# Getting the data from Google Drive

In [ ]:
!ls "/content/drive/My Drive/Colab Notebooks/Solution_IIT_Jodhpur/GurNum/"

train  val


In [ ]:
base_dir = "/content/new_dir"
train_dir = os.path.join(base_dir, 'train/')
val_dir = os.path.join(base_dir, 'val/')


In [ ]:
## Rough check if image is grayscale or rgb
from PIL import Image

def is_grey_scale(img_path):

  img = Image.open(img_path).convert('RGB')
  w,h = img.size
  for i in range(w):
      for j in range(h):
          r,g,b = img.getpixel((i,j))
          if r != g != b: return False
  return True

In [ ]:
is_grey_scale(img_path)

Check Size of the Image

In [ ]:
from PIL import Image
img_path = os.path.join(train_dir, '0/1.bmp')
im = Image.open(img_path)
width, height = im.size

In [ ]:
width, height

(32, 32)

# Data Pipeline and Preprocessing

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
train_generator = train_datagen.flow_from_directory(
    train_dir, 
    target_size=(32,32), 
    color_mode='grayscale',
    batch_size=32,
    shuffle=True,
    subset="training"
)



Found 800 images belonging to 10 classes.


In [ ]:
validation_generator = train_datagen.flow_from_directory(
    train_dir, 
    target_size=(32,32), 
    color_mode='grayscale',
    batch_size=16,
    subset="validation"
)


Found 200 images belonging to 10 classes.


In [ ]:
test_generator = test_datagen.flow_from_directory(
    val_dir,
    target_size=(32,32), 
    color_mode='grayscale'
)

Found 178 images belonging to 10 classes.


/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/utils.py:179: UserWarning: Using ".tiff" files with multiple bands will cause distortion. Please verify your output.
  warnings.warn('Using ".tiff" files with multiple bands '


Display Images from Train and Validation  generator

In [ ]:
temp_x = train_generator._get_batches_of_transformed_samples([1])[0]       # choosing only data point at index 1
temp_y = train_generator._get_batches_of_transformed_samples([1])[1]

# plt.imshow(temp_x[0])
plt.imshow(tf.squeeze(temp_x[0]),cmap="gray")
print("Label = {}".format(temp_y))
#print(x_train[5,:,:,:])

# Augmented Data Pipeline


In [ ]:
train_datagen = ImageDataGenerator(
      rescale=1./255,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      fill_mode='nearest', 
      validation_split=0.2
)

In [ ]:
au_train_generator = train_datagen.flow_from_directory(
    train_dir, 
    target_size=(32,32), 
    color_mode='grayscale',
    batch_size=32,
    shuffle=True,
    subset="training"
)


au_validation_generator = train_datagen.flow_from_directory(
    train_dir, 
    target_size=(32,32), 
    color_mode='grayscale',
    batch_size=16,
    subset="validation"
)

Found 800 images belonging to 10 classes.
Found 200 images belonging to 10 classes.


# Create Model

In [ ]:
## Model Definition

model = tf.keras.models.Sequential([tf.keras.layers.Flatten(), 

                                    tf.keras.layers.Dense(256, activation=tf.nn.relu),                                    
                                    tf.keras.layers.Dense(64, activation=tf.nn.relu),
                                    
                                    tf.keras.layers.Dense(10, activation=tf.nn.softmax)])

# Training The Model 


## On Normal Dataset

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint

filepath = 'm1_model.hdf5'
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=0,
                             save_best_only=True, save_weights_only=False,
                             mode='auto')


In [ ]:
model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss = 'categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.fit(train_generator, validation_data=validation_generator , epochs=50, callbacks=[checkpoint])

In [ ]:
model = tf.keras.models.load_model(filepath)

In [ ]:
### Evaluations
results_1 = model.evaluate(train_generator, verbose=0)
print(f" Training \t Loss={results_1[0]} \t Accuracy ={results_1[1]}")

results_2 = model.evaluate(validation_generator , verbose=0)
print(f" Validation \t Loss={results_2[0]} \t Accuracy ={results_2[1]}")

results_3 = model.evaluate(test_generator , verbose=0)
print(f" Test \t Loss={results_3[0]} \t Accuracy ={results_3[1]}")

 Training 	 Loss=0.009357860311865807 	 Accuracy =1.0
 Validation 	 Loss=0.04819011315703392 	 Accuracy =0.9950000047683716
 Test 	 Loss=0.20385056734085083 	 Accuracy =0.9438202381134033


## On Augmented Dataset


In [ ]:
model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=0.001),
              loss = 'categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.fit(au_train_generator, validation_data=au_validation_generator , epochs=30)

In [ ]:
results = model.evaluate(test_generator)

6/6 [==============================] - 0s 13ms/step - loss: 0.3630 - accuracy: 0.9213


## Save Model With Best Validation Loss

In [ ]:
model_min = tf.keras.models.Sequential([tf.keras.layers.Flatten(), 
                                    
                                    
                                    tf.keras.layers.Dense(128, activation=tf.nn.relu),
                                    
                                    tf.keras.layers.Dense(10, activation=tf.nn.softmax)])

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint

filepath = 'new_model_6.hdf5'
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=0,
                             save_best_only=True, save_weights_only=False,
                             mode='auto')


In [ ]:
model_min.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=0.001),
              loss = 'categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model_min.fit(train_generator, validation_data=validation_generator , epochs=20, callbacks=[checkpoint])

In [ ]:
model_min = tf.keras.models.load_model(filepath)

In [ ]:
results = model_min.evaluate(test_generator)

6/6 [==============================] - 0s 15ms/step - loss: 0.1962 - accuracy: 0.9551


In [ ]:
results = model_min.evaluate(train_generator)

25/25 [==============================] - 0s 9ms/step - loss: 0.0073 - accuracy: 1.0000


In [ ]:
results = model_min.evaluate(validation_generator)

13/13 [==============================] - 0s 6ms/step - loss: 0.0430 - accuracy: 0.9850


## Model With BatchNorm and Dropout

In [ ]:
bn_model = tf.keras.models.Sequential([tf.keras.layers.Flatten(), 
                                    
                                    
                                    tf.keras.layers.Dense(512, activation=tf.nn.relu),
                                    tf.keras.layers.BatchNormalization(),
                                    

                                    tf.keras.layers.Dense(256, activation=tf.nn.relu),
                                    tf.keras.layers.BatchNormalization(),
                                    
                                    
                                    tf.keras.layers.Dense(10, activation=tf.nn.softmax)])

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint

filepath = 'm2_model.hdf5'
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=0,
                             save_best_only=True, save_weights_only=False,
                             mode='auto')


In [ ]:
bn_model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss = 'categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
bn_model.fit(train_generator, validation_data=validation_generator , epochs=50, callbacks=[checkpoint])

In [ ]:
bn_model = tf.keras.models.load_model(filepath)

In [ ]:
### Evaluations
results_1 = bn_model.evaluate(train_generator, verbose=0)
print(f" Training \t Loss={results_1[0]} \t Accuracy ={results_1[1]}")

results_2 = bn_model.evaluate(validation_generator , verbose=0)
print(f" Validation \t Loss={results_2[0]} \t Accuracy ={results_2[1]}")

results_3 = bn_model.evaluate(test_generator , verbose=0)
print(f" Test \t Loss={results_3[0]} \t Accuracy ={results_3[1]}")

 Training 	 Loss=0.001028865692205727 	 Accuracy =1.0
 Validation 	 Loss=0.06172638386487961 	 Accuracy =0.9750000238418579
 Test 	 Loss=0.21978290379047394 	 Accuracy =0.9382022619247437


# Add Regularization

## L2 Regularization

### BatchNorm Model

In [ ]:
bn_model = tf.keras.models.Sequential([tf.keras.layers.Flatten(), 
                                    
                                    
                                    tf.keras.layers.Dense(512, activation=tf.nn.relu, kernel_regularizer=tf.keras.regularizers.l2(l=0.01)),
                                    tf.keras.layers.BatchNormalization(),
                                    

                                    tf.keras.layers.Dense(256, activation=tf.nn.relu, kernel_regularizer=tf.keras.regularizers.l2(l=0.01)),
                                    tf.keras.layers.BatchNormalization(),
                                    
                                    
                                    tf.keras.layers.Dense(10, activation=tf.nn.softmax)])

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint

filepath = 'm3_model.hdf5'
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=0,
                             save_best_only=True, save_weights_only=False,
                             mode='auto')


In [ ]:
bn_model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss = 'categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
bn_model.fit(train_generator, validation_data=validation_generator , epochs=300, callbacks=[checkpoint])

In [ ]:
bn_model = tf.keras.models.load_model(filepath)

In [ ]:
### Evaluations
results_1 = bn_model.evaluate(train_generator, verbose=0)
print(f" Training \t Loss={results_1[0]} \t Accuracy ={results_1[1]}")

results_2 = bn_model.evaluate(validation_generator , verbose=0)
print(f" Validation \t Loss={results_2[0]} \t Accuracy ={results_2[1]}")

results_3 = bn_model.evaluate(test_generator , verbose=0)
print(f" Test \t Loss={results_3[0]} \t Accuracy ={results_3[1]}")

 Training 	 Loss=0.01904480718076229 	 Accuracy =1.0
 Validation 	 Loss=0.0352955237030983 	 Accuracy =1.0
 Test 	 Loss=0.17472682893276215 	 Accuracy =0.9550561904907227


### Dropout Model

In [ ]:
drop_model = tf.keras.models.Sequential([tf.keras.layers.Flatten(), 
                                    
                                    tf.keras.layers.Dense(512, activation=tf.nn.relu, kernel_regularizer=tf.keras.regularizers.l2(l=0.01)),
                                    tf.keras.layers.Dropout(0.5),
                                    

                                    tf.keras.layers.Dense(256, activation=tf.nn.relu, kernel_regularizer=tf.keras.regularizers.l2(l=0.01)),
                                    tf.keras.layers.Dropout(0.5),
                                    
                                    
                                    tf.keras.layers.Dense(10, activation=tf.nn.softmax)])

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint

filepath = 'm4_model.hdf5'
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=0,
                             save_best_only=True, save_weights_only=False,
                             mode='auto')


In [ ]:
drop_model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss = 'categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
drop_model.fit(train_generator, validation_data=validation_generator , epochs=300, callbacks=[checkpoint])

In [ ]:
drop_model = tf.keras.models.load_model(filepath)

In [ ]:
### Evaluations
results_1 = drop_model.evaluate(train_generator, verbose=0)
print(f" Training \t Loss={results_1[0]} \t Accuracy ={results_1[1]}")

results_2 = drop_model.evaluate(validation_generator , verbose=0)
print(f" Validation \t Loss={results_2[0]} \t Accuracy ={results_2[1]}")

results_3 = drop_model.evaluate(test_generator , verbose=0)
print(f" Test \t Loss={results_3[0]} \t Accuracy ={results_3[1]}")

 Training 	 Loss=0.1442180871963501 	 Accuracy =1.0
 Validation 	 Loss=0.1678999960422516 	 Accuracy =0.9850000143051147
 Test 	 Loss=0.311099648475647 	 Accuracy =0.9550561904907227


### Batch Norm + Dropout

In [ ]:
bn_drop_model = tf.keras.models.Sequential([tf.keras.layers.Flatten(), 
                                    
                                    tf.keras.layers.Dense(512, activation=tf.nn.relu, kernel_regularizer=tf.keras.regularizers.l2(l=0.01)),
                                    tf.keras.layers.Dropout(0.5),
                                    tf.keras.layers.BatchNormalization(),
                                    

                                    tf.keras.layers.Dense(256, activation=tf.nn.relu, kernel_regularizer=tf.keras.regularizers.l2(l=0.01)),
                                    tf.keras.layers.Dropout(0.5),
                                    tf.keras.layers.BatchNormalization(),
                                    
                                    
                                    tf.keras.layers.Dense(10, activation=tf.nn.softmax)])

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint

filepath = 'm4_model.hdf5'
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=0,
                             save_best_only=True, save_weights_only=False,
                             mode='auto')


In [ ]:
bn_drop_model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss = 'categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
bn_drop_model.fit(train_generator, validation_data=validation_generator , epochs=300, callbacks=[checkpoint])

In [ ]:
bn_drop_model = tf.keras.models.load_model(filepath)

In [ ]:
### Evaluations
results_1 = bn_drop_model.evaluate(train_generator, verbose=0)
print(f" Training \t Loss={results_1[0]} \t Accuracy ={results_1[1]}")

results_2 = bn_drop_model.evaluate(validation_generator , verbose=0)
print(f" Validation \t Loss={results_2[0]} \t Accuracy ={results_2[1]}")

results_3 = bn_drop_model.evaluate(test_generator , verbose=0)
print(f" Test \t Loss={results_3[0]} \t Accuracy ={results_3[1]}")

 Training 	 Loss=0.07568126916885376 	 Accuracy =1.0
 Validation 	 Loss=0.12537124752998352 	 Accuracy =0.9800000190734863
 Test 	 Loss=0.34289267659187317 	 Accuracy =0.9382022619247437


# Final Model

bn_drop_model (1024 -> 512 -> 256 -> 10)
- With L2 Regularization 
- Dropout(0.5) + Batch Normalization For Generalizing

## Data For Final Model 
  
Trained on all of Training Dataset 
and Tested on Validation Dataset

In [ ]:
train_datagen_final = ImageDataGenerator(rescale=1./255)

train_generator_final = train_datagen_final.flow_from_directory(
    train_dir, 
    target_size=(32,32), 
    color_mode='grayscale',
    batch_size=32,
    shuffle=True
)

test_datagen_final = ImageDataGenerator(rescale=1./255)
test_generator_final = test_datagen_final.flow_from_directory(
    val_dir,
    target_size=(32,32), 
    color_mode='grayscale'
)

Found 1000 images belonging to 10 classes.
Found 178 images belonging to 10 classes.


/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/utils.py:179: UserWarning: Using ".tiff" files with multiple bands will cause distortion. Please verify your output.
  warnings.warn('Using ".tiff" files with multiple bands '


In [ ]:
## Model Definition

bn_drop_model = tf.keras.models.Sequential([tf.keras.layers.Flatten(), 
                                    
                                    tf.keras.layers.Dense(512, activation=tf.nn.relu, kernel_regularizer=tf.keras.regularizers.l2(l=0.01)),
                                    tf.keras.layers.Dropout(0.5),
                                    tf.keras.layers.BatchNormalization(),
                                    

                                    tf.keras.layers.Dense(256, activation=tf.nn.relu, kernel_regularizer=tf.keras.regularizers.l2(l=0.01)),
                                    tf.keras.layers.Dropout(0.5),
                                    tf.keras.layers.BatchNormalization(),
                                    
                                    
                                    tf.keras.layers.Dense(10, activation=tf.nn.softmax)])

In [ ]:
## For Saving Model With Best Validation Loss

from tensorflow.keras.callbacks import ModelCheckpoint

final_model = 'final_model.hdf5'
checkpoint = ModelCheckpoint(final_model, monitor='val_loss', verbose=0,
                             save_best_only=True, save_weights_only=False,
                             mode='auto')

In [ ]:
bn_drop_model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss = 'categorical_crossentropy',
              metrics=['accuracy'])

Epoch 1/2
32/32 [==============================] - 0s 7ms/step - loss: 12.9122 - accuracy: 0.1390
Epoch 2/2
32/32 [==============================] - 0s 7ms/step - loss: 12.0358 - accuracy: 0.2920


In [ ]:
bn_drop_model.fit(train_generator, validation_data=test_generator_final , epochs=300, callbacks=[checkpoint])

Epoch 1/300
25/25 [==============================] - 0s 18ms/step - loss: 12.8721 - accuracy: 0.1500 - val_loss: 12.0310 - val_accuracy: 0.3876
Epoch 2/300
25/25 [==============================] - 0s 13ms/step - loss: 12.1695 - accuracy: 0.2800 - val_loss: 11.5710 - val_accuracy: 0.6461
Epoch 3/300
25/25 [==============================] - 0s 13ms/step - loss: 11.5801 - accuracy: 0.4313 - val_loss: 11.2033 - val_accuracy: 0.7022
Epoch 4/300
25/25 [==============================] - 0s 14ms/step - loss: 11.1935 - accuracy: 0.5250 - val_loss: 10.9018 - val_accuracy: 0.7697
Epoch 5/300
25/25 [==============================] - 0s 13ms/step - loss: 10.8923 - accuracy: 0.5913 - val_loss: 10.6497 - val_accuracy: 0.7809
Epoch 6/300
25/25 [==============================] - 0s 13ms/step - loss: 10.6204 - accuracy: 0.6550 - val_loss: 10.3880 - val_accuracy: 0.8202
Epoch 7/300
25/25 [==============================] - 0s 14ms/step - loss: 10.3422 - accuracy: 0.7362 - val_loss: 10.1504 - val_accuracy:

In [ ]:
bn_drop_model = tf.keras.models.load_model(filepath)

In [ ]:
### Evaluations
results_1 = bn_drop_model.evaluate(train_generator, verbose=0)
print(f" Training \t Loss={results_1[0]} \t Accuracy ={results_1[1]}")

results_2 = bn_drop_model.evaluate(validation_generator , verbose=0)
print(f" Validation \t Loss={results_2[0]} \t Accuracy ={results_2[1]}")


 Training 	 Loss=0.17333807051181793 	 Accuracy =1.0
 Validation 	 Loss=0.2028772532939911 	 Accuracy =0.9850000143051147


## For Evaluation On Test Side



In [ ]:
test_dir = '' ## Insert the path of file containing the test set images


In [ ]:
test_datagen_eval = ImageDataGenerator(rescale=1./255)
test_generator_eval = test_datagen_eval.flow_from_directory(
    test_dir,
    target_size=(32,32), 
    color_mode='grayscale'
)

In [ ]:
## Load Final Model
bn_drop_model = tf.keras.models.load_model(final_model)

In [ ]:
## Gives Loss and Accuracy on the Actual Test Side
results_final = bn_drop_model.evaluate(test_generator_eval, verbose=0)
print(f" Testing \t Loss={results_final[0]} \t Accuracy ={results_final[1]}")